In [2]:
import pickle
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
def carregar_modelo():
    path = '../resources/modelos/modelo_classificacao_svc'
    try:
        modelo = pickle.load(open(path, 'rb'))
        print('Modelo carregado com sucesso!\n')

        return modelo
    except FileNotFoundError:
        print('O arquivo não foi encontrado')
    except IOError:
        print('Erro ao carregar arquivo')
        
        
def criar_embeddings(df_tweets):
    try:
        nlp = spacy.load('pt_core_news_lg')
    except (IOError, OSError):
        download('pt_core_news_lg')
        nlp = spacy.load('pt_core_news_lg')
    # desativamos todos os outros pipes que vem com o modelo nlp porque não preicsaremos deles
    with nlp.disable_pipes():
        # transformamos cada texto em um vetor e colocamos em uma array
        print('Fazendo os word embeddings')
        vetores = np.array([nlp(texto).vector for texto in df_tweets.Texto])

    return vetores

In [4]:
path_datasets = '../resources/datasets'

# importação dos dados
df_treinamento = pd.read_csv(
    f'{path_datasets}/tweets_ekman.csv',
    usecols=['Texto', 'Sentimento']
).dropna()
df_alvo = pd.read_csv(f'{path_datasets}/tweets_pandemia.csv')

x_train, x_test, y_train, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Sentimento'],
    test_size=0.2,
    random_state=42
)

# processamento dos dados para word embeddings
embeddings = criar_embeddings(x_train)
embeddings_teste = criar_embeddings(x_test)

Fazendo os word embeddings
Fazendo os word embeddings


In [5]:
modelo_svc = carregar_modelo()
previsoes = modelo_svc.predict(embeddings_teste)

D:\home\Desenvolvimento\Analise-de-sentimentos-pandemia-covid19\emocoes_venv\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 0.22.2.post1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Modelo carregado com sucesso!


In [6]:
# testar performance
print('Relatório Linear SVC')
print(classification_report(y_test, previsoes))

Relatório Linear SVC
              precision    recall  f1-score   support

      Neutro       0.73      0.95      0.83       495
       feliz       0.63      0.62      0.62      1931
        medo       0.53      0.39      0.45      2320
        nojo       0.49      0.61      0.54      1648
       raiva       0.17      0.40      0.24       562
      triste       0.51      0.35      0.42      2097

    accuracy                           0.50      9053
   macro avg       0.51      0.55      0.52      9053
weighted avg       0.53      0.50      0.50      9053
